# SAT/SMT Solvers

Understanding the basics of how Z3 works helps in formulating problems that the solver can handle effectively.

Z3 and other SMT solvers are based on SAT (satisfiability problem) theory. SMT extends SAT by using predicates from various theories (numbers, arrays, bit-vectors, etc.), making them versatile tools for solving complex logical formulas.

### SAT

SAT determines the satisfiability of propositional logic formulas.

A propositional logic formula is a combination of variables, True/False values, and operators (And, Or, Implies, Not). By assigning values to the variables, the formula outputs true or false, akin to logic circuits.

### SMT

SMT extends SAT by using predicates from various theories (numbers, arrays, bit-vectors). The key question is whether there exists an assignment of values that makes the formula true. SMT solvers search for such assignments rather than computing a final value. This task is in the NP complexity class, meaning the search space is exponential, but each solution can be quickly verified.

SMT solvers use advanced heuristics and techniques such as backjumping, two watched literals, conflict-directed clause learning, and random restarts to efficiently navigate the solution space.

### SMT Solver Structure

SMT solvers operate on two levels: handling complex logical structures and dealing with numbers, formulas, etc. Ideally, specific domain facts are represented as Boolean variables, and the SMT solver finds satisfying assignments. In practice, specialized solvers for different mathematical domains are used to interpret conditions and solve parts of the problem, ensuring satisfiability within the underlying theories. This integration of solving techniques with domain-specific theories is the essence of SMT (Satisfiability Modulo Theories).

---

Lets take a look on some examples.

## 1. Pigeonholes

Prove that if you try to put `n+1` pigeons into `n` pigeonholes, at least one pigeonhole will contain more than one pigeon.


In [32]:
from z3 import *



n = Int('n')
n_plus_one_pigeons = n + 1

# these are magic constants to limit calculations below,
# either way z3 will hang forever.
PIGEONHOLES = 10
PIGEONS = PIGEONHOLES + 1
# mapper pigeon to pigeonhole
pigeonhole = Function('pigeonhole', IntSort(), IntSort())

constraints = [n > 0]
for i in range(PIGEONS):  # Up to n + 1 pigeons
    constraints.append(Implies(i < n_plus_one_pigeons, And(pigeonhole(i) >= 0, pigeonhole(i) < n)))

# Each pigeonhole can hold at most one pigeon (this needs to be enforced)
at_least_one_pigeonhole = Or([Sum([If(pigeonhole(j) == i, 1, 0) for j in range(PIGEONS)]) > 1 for i in range(PIGEONHOLES)])

s = Solver()
s.add(n == PIGEONHOLES)  
s.add(*constraints)
s.add(Not(at_least_one_pigeonhole)) 

if s.check() == sat:
    print("Counterexample found:")
    print(s.model())
else:
    print("Theorem holds.")


Theorem holds.


In [ ]:
## 2. Proving the Correctness of Dijkstra's Algorithm properties

In [34]:
from z3 import *

# graph and distances
n = 5
graph = Function('graph', IntSort(), IntSort(), RealSort())
dist = Function('dist', IntSort(), RealSort())
src = Int('src')
tgt = Int('tgt')

# shortest path condition
shortest_path = ForAll([src, tgt], Implies(And(src != tgt, src >= 0, src < n, tgt >= 0, tgt < n), 
                dist(tgt) <= dist(src) + graph(src, tgt)))

s = Solver()
s.add(shortest_path)

if s.check() == sat:
    print("ok.")
else:
    print("not ok.")


ok.


In [ ]:
## 3. The Fundamental Theorem of Arithmetic

In [44]:
from z3 import *

a = Int('a')
b = Int('b')
c = Int('c')
d = Int('d')
n = Int('n')


# clause that a number is prime
def is_prime(x):
    return And(x > 1, ForAll([a, b], Implies(a * b == x, Or(a == 1, b == 1))))

# if two factorizations of n are the same, the sets of prime factors are identical
unique_factorization = ForAll([c, d], Implies(And(c * d == n, is_prime(c), is_prime(d)), Exists([a, b], And(a * b == n, is_prime(a), is_prime(b), a == c, b == d))))

s = Solver()
s.add(n > 1)
s.add(unique_factorization)

if s.check() == sat:
    print("The world is working.")
    print(s.model())
else:
    print("We are doomed, or you screwed in the goal spec.")


The world is working.
[n = 2]




## 4. Proving Cantor's Diagonalization Argument

Description: Prove that the set of all real numbers between 0 and 1 is uncountable using Cantor's diagonalization argument. This theorem shows that for any supposed list of all real numbers, we can construct a real number not on the list.

### Cantor's Diagonalization Argument

1. Assume that we have a list of all real numbers between 0 and 1.
2. Construct a new real number by changing the *i*-th digit of the *i*-th number on the list.
3. This new number cannot be on the list, as it differs from the *i*-th number in the *i*-th digit.

We'll demonstrate this idea by proving that no list of real numbers can contain all real numbers between 0 and 1.

In [50]:
from z3 import *

n = 10  # sequense len var
reals = [Real(f'r_{i}') for i in range(n)]

digits = [[Int(f'd_{i}_{j}') for j in range(n)] for i in range(n)]
digit_constraints = [And(digits[i][j] >= 0, digits[i][j] <= 9) for i in range(n) for j in range(n)]

# some a new real number not in the list
new_digits = [If(digits[i][i] == 9, 0, digits[i][i] + 1) for i in range(n)]
new_real = Sum([new_digits[i] * 10**(-i-1) for i in range(n)])

# the new real number is diff from each number in the list
differences = [Or([digits[i][j] != new_digits[j] for j in range(n)]) for i in range(n)]

s = Solver()
s.add(digit_constraints)
s.add(differences)

if s.check() == sat:
    print("Cantor's diagonalization argument holds: The set is uncountable.")
    #print(s.model())
else:
    print("The set is countable (unexpected result).")


Cantor's diagonalization argument holds: The set is uncountable.
